# Generate inteferograms without the need to download OPERA CSLC-S1 products locally
## This tutorial demonstrates how to query and work with the OPERA CSLC-S1 provisional data products from AWS S3 bucket.

---    

### Data Used in the Example:   

- **10 meter (Northing) x 5 meter (Easting) North America OPERA Coregistered Single Look Complex from Sentinel-1 provisional product**
    - This dataset contains Level-2 OPERA provisional coregistered single-look-complex (CSLC) data from Sentinel-1 (S1). <span style="color:red"> The data in this example are provisional, non-validated, geocoded CSLC-S1 data covering the 2019 Ridgecrest earthquake in California</span>. 
    
    - The OPERA project is generating geocoded burst-wise CSLC-S1 products over North America which includes USA and US Territories within 200 km from the US border, Canada, and all mainland countries from the southern US border down to and including Panama. Each pixel within a burst SLC is represented by a complex number and contains both the amplitude and phase information. The CSLC-S1 products are distributed over projected map coordinates using the Universal Transverse Mercator (UTM) projection with spacing in the X- and Y-directions of 5 m and 10 m, respectively. Each OPERA CSLC-S1 product is distributed as a HDF5 file following the CF-1.8 convention with separate groups containing the data raster layers, the low-resolution correction layers, and relevant product metadata.

    - For more information about the OPERA project and other products please visit our website at https://www.jpl.nasa.gov/go/opera .

- **Science Dataset (SDS) layer: Ridgecrest Earthquakes**
    - An earthquake sequence occurred near Ridgecrest, CA that consists of two mainshocks: Mw 6.4 on July 4, 2019, and 7.1 on July 5, 2019 (local time), with numerous aftershocks. This was the largest earthquake in Southern California since the 1999 Hetcor Mine earthquake [[USGS](https://earthquake.usgs.gov/storymap/index-ridgecrest.html)]. Here, we demonstrate how to create an interferogram and calculate for the interferometric coherence using pre-event (July 04, 2019) and after event (July 16, 2019) CSLC-S1 products. 

Please refer to the [OPERA Product Specification Document](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_CSLC_URS314199.pdf) for details about the CSLC-S1 product.

*Prepared by M. Grace Bato and K. Devlin*

---
## 1. Getting Started <a id="getstarted"></a>

### 1.1 Import Packages <a id="1.1"></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import glob
import h5py
import fsspec
from pyproj import Proj, CRS
import matplotlib.pyplot as plt
import shapely.wkt as wkt
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio import merge
import folium
from folium import plugins
import rioxarray
import matplotlib as mpl
import sys
sys.path.append('../../')
from src.cslc_utils import get_s3path, read_cslc, cslc_info, rasterWrite, custom_merge, colorize, getbasemaps, moving_window_mean

import warnings
warnings.filterwarnings('ignore')

In [8]:
path1 = 's3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190704T135212Z_v0.1_20230708T134145Z/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190704T135212Z_v0.1_20230708T134145Z.h5'
path2 = 's3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190716T135212Z_v0.1_20230708T134123Z/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190716T135212Z_v0.1_20230708T134123Z.h5'
path3 = 's3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190704T135209Z_v0.1_20230708T134145Z/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190704T135209Z_v0.1_20230708T134145Z.h5'
path4 = 's3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190716T135210Z_v0.1_20230708T134123Z/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190716T135210Z_v0.1_20230708T134123Z.h5'

In [4]:
ids = ['T071-151220-IW2','T071-151219-IW2']

In [9]:
cslc1 = read_cslc(path1)
cslc2 = read_cslc(path2)

xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(path1)

ifg = cslc1*np.conj(cslc2)
phase = np.angle(ifg)

# matplotlib plots and save to gif
# Save each interferogram's phase as GeoTiff
transform = from_origin(xcoor[0],ycoor[0],dx,np.abs(dy))
rasterWrite(f'complex_ifg_{ids[0]}.tif',phase,transform,epsg,dtype='float32')

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190704T135212Z_v0.1_20230708T134145Z/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190704T135212Z_v0.1_20230708T134145Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190716T135212Z_v0.1_20230708T134123Z/OPERA_L2_CSLC-S1A_IW_T071-151220-IW2_VV_20190716T135212Z_v0.1_20230708T134123Z.h5


In [7]:
cslc1 = read_cslc(path3)
cslc2 = read_cslc(path4)

xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(path3)

ifg = cslc1*np.conj(cslc2)
phase = np.angle(ifg)

# Save each interferogram's phase as GeoTiff
transform = from_origin(xcoor[0],ycoor[0],dx,np.abs(dy))
rasterWrite(f'complex_ifg_{ids[1]}.tif',phase,transform,epsg,dtype='float32')

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190704T135209Z_v0.1_20230708T134145Z/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190704T135209Z_v0.1_20230708T134145Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190716T135210Z_v0.1_20230708T134123Z/OPERA_L2_CSLC-S1A_IW_T071-151219-IW2_VV_20190716T135210Z_v0.1_20230708T134123Z.h5
